In [2]:
import geopandas as gpd
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

def normalize_val(val):
    if pd.isnull(val):
        return ""
    return str(val).strip()

def normalize_df(df):
    df_norm = df.copy()
    for col in df_norm.columns:
        df_norm[col] = df_norm[col].apply(normalize_val)
    return df_norm

# --- 파일 경로 입력 ---
past_file = r"D:\test_jeju\check_uid\2024"
curr_file = r"D:\test_jeju\check_uid\2025"
input_folder = os.path.dirname(curr_file)

key_cols = ["UID", "O_UID"]   # 복합키

# --- 비교할 주요 컬럼 (실제 데이터 상황에 맞게 수정 가능) ---
target_cols = [
    "ID", "UID", "O_UID", "CLSF_NM", "CLSF_CD", "STDG_CD", "STDG_ADDR", "PNU",
    "LDCG_CD", "SB_PNU", "SB_LDCG_CD","SOURCE_NM", 'ER_DCLR_TP','COV_INFO',"검수"   # '검수'도 포함해야 merge 후 _올해 컬럼 사용 가능
]

# --- 데이터 불러오기 및 표준화 ---
df_past = gpd.read_file(past_file, encoding='cp949')
df_curr = gpd.read_file(curr_file, encoding='cp949')
df_past_norm = normalize_df(df_past)
df_curr_norm = normalize_df(df_curr)

# --- 올해 미갱신(검수 null/공백)만 추출 ---
if "검수" in df_curr_norm.columns:
    df_curr_null = df_curr_norm[df_curr_norm["검수"] == ""].copy()
else:
    df_curr_null = df_curr_norm.copy()

# --- 복합키별 강제 중복 제거(올해, 작년 모두) ---
df_curr_null = df_curr_null.drop_duplicates(subset=key_cols, keep='first')
df_past_norm = df_past_norm.drop_duplicates(subset=key_cols, keep='first')

# --- 올해 복합키 set 기준, 작년에서 해당 복합키 있는 행만 추출 (역시 중복 제거) ---
multi_key_tuples = set(tuple(x) for x in df_curr_null[key_cols].values)
df_past_matched = df_past_norm[df_past_norm[key_cols].apply(tuple, axis=1).isin(multi_key_tuples)].copy()
df_past_matched = df_past_matched.drop_duplicates(subset=key_cols, keep='first')

# --- 실제 두 데이터에 모두 존재하는 컬럼만 자동 선별 ---
past_cols = set(df_past_matched.columns)
curr_cols = set(df_curr_null.columns)
actual_cols = [col for col in target_cols if col in past_cols and col in curr_cols]
if len(actual_cols) < len(target_cols):
    print("다음 컬럼은 실제 데이터에 없으므로 제외됩니다:", set(target_cols) - set(actual_cols))
print("최종 비교 컬럼:", actual_cols)

# --- merge: 복합키 기준으로 실제 비교 컬럼만 붙이기 ---
df_merged = pd.merge(
    df_past_matched[actual_cols],
    df_curr_null[actual_cols],
    on=key_cols,
    suffixes=("_작년", "_올해"),
    how='right' # 올해 기준
)

# --- merge 후 중복 체크 ---
print("merge 후 전체 row수:", len(df_merged))
print("merge 후 복합키 유니크 수:", len(df_merged[key_cols].drop_duplicates()))

# --- 완전일치(모든 비교 컬럼 값 동일) 여부 함수 ---
def is_equal(row):
    for col in actual_cols:
        if col in key_cols or col == "검수":
            continue
        if row[f"{col}_작년"] != row[f"{col}_올해"]:
            return False  # 하나라도 다르면 False
    return True  # 모두 같으면 True

df_merged['완전일치'] = df_merged.apply(is_equal, axis=1)

# --- 어떤 컬럼이 달라졌는지, 그리고 어떻게 달라졌는지 상세 기록 ---
def diff_columns(row):
    return ", ".join([
        col for col in actual_cols
        if col not in key_cols and col != "검수" and row[f"{col}_작년"] != row[f"{col}_올해"]
    ])

def diff_detail(row):
    changes = []
    for col in actual_cols:
        if col in key_cols or col == "검수":
            continue
        v1 = row[f"{col}_작년"]
        v2 = row[f"{col}_올해"]
        if v1 != v2:
            changes.append(f"{col}: '{v1}' → '{v2}'")
    return "; ".join(changes)

df_merged['다른컬럼'] = df_merged.apply(diff_columns, axis=1)
df_merged['달라진_내용'] = df_merged.apply(diff_detail, axis=1)

# --- '검수' 컬럼명 (merge시 '_올해' suffix) ---
검수_컬럼명 = "검수_올해" if "검수_올해" in df_merged.columns else "검수"

# --- 불일치(=하나라도 다르면) & 올해 검수값이 1이 아닌 경우만 추출 ---
df_unmatched = df_merged[
    (~df_merged['완전일치']) &
    (df_merged[검수_컬럼명].fillna("").astype(str).str.strip() != "1")
]

# --- 저장 ---
out_file = os.path.join(input_folder, "UID_OUID_동일_불일치_검수미처리_row.csv")
df_unmatched.to_csv(out_file, index=False, encoding='utf-8-sig')

print(f"[결과] 값 불일치+올해 검수 1이 아닌 행: {len(df_unmatched)}건")
print(f"결과 파일: {out_file}")


최종 비교 컬럼: ['ID', 'UID', 'O_UID', 'CLSF_NM', 'CLSF_CD', 'STDG_CD', 'STDG_ADDR', 'PNU', 'LDCG_CD', 'SB_PNU', 'SB_LDCG_CD', 'SOURCE_NM', 'ER_DCLR_TP', 'COV_INFO', '검수']
merge 후 전체 row수: 137254
merge 후 복합키 유니크 수: 137254
[결과] 값 불일치+올해 검수 1이 아닌 행: 26건
결과 파일: D:\test_jeju\check_uid\UID_OUID_동일_불일치_검수미처리_row.csv
